In [ ]:
using CSV
using DataFrames
using LinearAlgebra
using Statistics
using Dates
using PlotlyJS

In [ ]:
function load_stocks(data_dir, stock_files)
    # List to store data from each file
    combined_df_list = DataFrame[]
    
    for file in stock_files
        path = joinpath(data_dir, file)
        
        if isfile(path)
            try
                # Load CSV
                df = CSV.read(path, DataFrame)
                
                # Keep only needed columns
                df = select(df, [:Date, :Ticker, :Close, :Sector])
                
                # Convert Date to DateTime
                df.Date = DateTime.(df.Date)
                
                push!(combined_df_list, df)
            catch e
                println("Error reading $file: $e")
            end
        else
            println("File not found: $file")
        end
    end
    
    # Concatenate all sector data
    if !isempty(combined_df_list)
        combined_df = vcat(combined_df_list...)
    else
        combined_df = DataFrame(Date=DateTime[], Ticker=String[], Close=Float64[], Sector=String[])
    end
    
    return combined_df
end

In [ ]:
# Configuration
data_dir = "C:/Users/Z2005081/Downloads/COPY FROM SCHOOL PC/DATA_MINNER/FULL_2020_2025_11_SECTION"
stock_files = [
    "Consumer_Discretionary.csv", "Consumer_Staples.csv", "Energy.csv",
    "Financials.csv", "Health_Care.csv", "Industrials.csv",
    "Information_Technology.csv", "Materials.csv", "Real_Estate.csv",
    "Communication_Services.csv", "Utilities.csv"
]

In [ ]:
# Load and prepare data
combined_df = load_stocks(data_dir, stock_files)

# Sort by Ticker and Date
sort!(combined_df, [:Ticker, :Date])

In [ ]:
# Calculate log returns
combined_df = transform(groupby(combined_df, :Ticker), 
    :Close => (x -> [missing; log.(x[2:end] ./ x[1:end-1])]) => :LogReturn
)

# Drop rows with missing log returns is commented out in original
# combined_df = dropmissing(combined_df, :LogReturn)

In [ ]:
# Display return
println("\nSample of cleaned stock log return data:\n")
println(combined_df)

In [ ]:
# Pivot: Tickers as columns, Dates as rows, LogReturn as values
ret_df = unstack(combined_df, :Date, :Ticker, :LogReturn)
sort!(ret_df, :Date)  # Sort rows by Date (just in case)
ret_df

In [ ]:
# Correlation Matrix
# First, get numeric columns (exclude Date column)
numeric_cols = names(ret_df)[2:end]  # Skip Date column
ret_matrix = Matrix(ret_df[:, numeric_cols])
corr_matrix = cor(ret_matrix, dims=1)

In [ ]:
# For correlation or covariance matrix
eigenvalues = eigvals(Symmetric(corr_matrix))

# Check if all eigenvalues are non-negative
is_psd = all(eigenvalues .>= -1e-8)  # Allow small numerical errors

println("Is the correlation matrix PSD? ", is_psd)

In [ ]:
println("Smallest eigenvalue: ", minimum(eigenvalues))

In [ ]:
# Cumulative Return for portfolio
cumsum(ret_matrix, dims=1)

In [ ]:
# Compute cumulative returns
cumulative_returns = cumsum(ret_matrix, dims=1)

# Create interactive figure
traces = GenericTrace[]
for (i, ticker) in enumerate(numeric_cols)
    push!(traces, scatter(
        x=ret_df.Date,
        y=cumulative_returns[:, i],
        mode="lines",
        name=ticker
    ))
end

# Customize layout
layout = Layout(
    title="Cumulative Log Returns (Interactive)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    width=1000,
    height=800,
    showlegend=false
)

# Show the interactive plot
plot(traces, layout)

In [ ]:
# Mean(return)
mean(ret_matrix, dims=1)

In [ ]:
# Std(risk)
std(ret_matrix, dims=1)

In [ ]:
# Weight
n_assets = size(ret_matrix, 2)
W = ones(n_assets) / n_assets  # create a vector of equal weights
W

In [ ]:
# Manual Expected Return
sum(W .* vec(mean(ret_matrix, dims=1)))

In [ ]:
# Expected(Average)Return
dot(vec(mean(ret_matrix, dims=1)), W)

In [ ]:
# Covariance Matrix
cov_matrix = cov(ret_matrix, dims=1)

In [ ]:
# Variance(risk) of the portfolio (550 by 550)
sqrt(W' * cov_matrix * W)

In [ ]:
# Sharpe Ratio for individual asset (Risk adjusted return in form of the sharpe ratio)
vec(mean(ret_matrix, dims=1)) ./ vec(std(ret_matrix, dims=1))

In [ ]:
# Sharpe Ratio of the portfolio
dot(vec(mean(ret_matrix, dims=1)), W) / sqrt(W' * cov_matrix * W)